In [7]:
from bs4 import BeautifulSoup
import requests
import json
import time

In [49]:
import sys

reload(sys)

sys.setdefaultencoding('utf-8')



NameError: name 'reload' is not defined

In [121]:
def youtubeCrawlLiveData(channelId):
    dataSet = {}
    url = "https://www.youtube.com"
    
    urldata = requests.get(url+'/channel/'+channelId)
    
    htmldata = BeautifulSoup(urldata.text, 'html.parser')
    link = htmldata.select('div.yt-lockup-dismissable')
    linkData=link[0].select('div.yt-lockup-content')
    dataLiveConfirm=linkData[0].select_one('a')['data-sessionlink']
    
    if dataLiveConfirm.find('live') > 0 :
        dataSet['_id'] = channelId
        dataSet['channelId'] = channelId
        imgData=link[0].select_one('div.yt-lockup-thumbnail > span > a > span > span > span > img')
        dataSet['imgDataSrc'] = imgData.attrs['data-thumb']
        liveData = link[0].select_one('div.yt-lockup-content > h3 > a')
        dataSet['liveDataHref'] = url+liveData.attrs['href']
        dataSet['liveDataTitle'] = liveData.attrs['title']
        creatorData = link[0].select_one('div.yt-lockup-content > div.yt-lockup-byline > a')
        dataSet['creatorDataHref'] = url+creatorData.attrs['href']
        dataSet['creatorDataName'] = creatorData.text
        AttdData = link[0].select_one('div.yt-lockup-content > div.yt-lockup-meta > ul > li ')
        dataSet['liveAttdc'] = AttdData.text.partition('명')[0]
        dataSet['platform'] = "Youtube"
        dataSet['onLive'] = True
    else :
        dataSet= {
            "_id" : channelId,
            "onLive" : False
        }
    return json.dumps(dataSet, ensure_ascii=False)



In [263]:
def twitchCrawlLiveData(channelId):
    dataSet = {}
    url = "http://api.twitch.tv/helix/streams?user_login="
    headers = {"Client-ID":"x61q9wk8c5hw5q00x49jzbesmmodn1"}
    urldata = requests.get(url+channelId, headers=headers)
    urlJsonData = json.loads(urldata.text)
    print(urlJsonData)
    if urlJsonData != {'data': [], 'pagination': {}} :
        dataSet['_id'] = channelId
        dataSet['channelId'] = urlJsonData['data'][0]['user_id']
        dataSet['imgDataSrc'] = urlJsonData['data'][0]['thumbnail_url'].replace('{width}', '1600').replace('{height}', '900')
        dataSet['liveDataHref'] = "http://twitch.tv/"+channelId
        dataSet['liveDataTitle'] = urlJsonData['data'][0]['title']
        dataSet['creatorDataHref'] = "http://twitch.tv/"+channelId
        dataSet['creatorDataName'] = urlJsonData['data'][0]['user_name']
        dataSet['liveAttdc'] = urlJsonData['data'][0]['viewer_count']
        dataSet['platform'] = "Twithch"
        dataSet['onLive'] = True
    else :
        dataSet= {
            "_id" : channelId,
            "onLive" : False
        }
    return json.dumps(dataSet, ensure_ascii=False)



In [264]:
def sendDataToDb(dataSet):
    url="http://localhost:4000/api/posts"
    headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
    r= requests.post(url, data=dataSet.encode('utf-8'), headers=headers)

In [265]:
dataList = ['heavyrainism', 'kimdoe', 'flurry1989', 'zilioner']

In [266]:
for i in dataList :
    a=twitchCrawlLiveData(i)
    sendDataToDb(a)

{'data': [], 'pagination': {}}
{'data': [{'id': '37707759376', 'user_id': '102845970', 'user_name': '김도', 'game_id': '510649', 'type': 'live', 'title': '사이버펑크 택배 CLOUDPUNK', 'viewer_count': 4528, 'started_at': '2020-04-24T05:59:08Z', 'language': 'ko', 'thumbnail_url': 'https://static-cdn.jtvnw.net/previews-ttv/live_user_kimdoe-{width}x{height}.jpg', 'tag_ids': ['ab2975e3-b9ca-4b1a-a93e-fb61a5d5c3a4']}], 'pagination': {}}
{'data': [{'id': '37705499232', 'user_id': '92371909', 'user_name': '플러리', 'game_id': '138585', 'type': 'live', 'title': 'ㅎㅎㅎㅎ하스스톤', 'viewer_count': 6342, 'started_at': '2020-04-24T02:56:57Z', 'language': 'ko', 'thumbnail_url': 'https://static-cdn.jtvnw.net/previews-ttv/live_user_flurry1989-{width}x{height}.jpg', 'tag_ids': ['ab2975e3-b9ca-4b1a-a93e-fb61a5d5c3a4', 'dbacccc8-57e7-4411-9664-9982f96d629c']}], 'pagination': {}}
{'data': [], 'pagination': {}}
